In [1]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer

from dwave.plugins.qiskit import DWaveMinimumEigensolver
from dwave.system import DWaveCliqueSampler, LeapHybridSampler

import numpy as np

# Convert integer linear program from qiskit and solve with DWaveCliqueSampler and Leaps Hybrid Solver

In [2]:
A = np.array([[0,1,2,3,4,1],
              [2,1,4,1,0,2],
              [1,5,2,3,1,1],
              [0,0,1,2,2,0]])
m, n = A.shape
c =  np.array([-1,2,-2,2,1,1])
b = np.array([6,8,5,2])

solution = np.array([2,0,0,0,1,2])

print(all(A.dot(solution) == b))

True


# Problem statement:
Given $A\in Z^{mxn}, b\in Z^{m}, c\in Z^{n}$ 

find $x\in N^n$ with:

$\min c^T x $ with constraint $Ax=b$ 

In [3]:
model = QuadraticProgram("Integer Test")
lowerbound = 0
upperbound = 7 

# add variables
x = [model.integer_var(lowerbound = lowerbound, upperbound = upperbound, name = 'x'+str(j)) for j in range(n)]

# add linear constraints
for i in range(m):
    model.linear_constraint(linear=A[i], sense='==', rhs=b[i], name='A'+str(i)+'eq'+'b'+str(i))
    
# add linear objective function

model.minimize(linear = c)

In [4]:
print(model.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Integer Test

Minimize
 obj: - x0 + 2 x1 - 2 x2 + 2 x3 + x4 + x5
Subject To
 A0eqb0: x1 + 2 x2 + 3 x3 + 4 x4 + x5 = 6
 A1eqb1: 2 x0 + x1 + 4 x2 + x3 + 2 x5 = 8
 A2eqb2: x0 + 5 x1 + 2 x2 + 3 x3 + x4 + x5 = 5
 A3eqb3: x2 + 2 x3 + 2 x4 = 2

Bounds
       x0 <= 7
       x1 <= 7
       x2 <= 7
       x3 <= 7
       x4 <= 7
       x5 <= 7

Generals
 x0 x1 x2 x3 x4 x5
End



# Solve with DWaveCliqueSampler

In [5]:
# Use D-Wave QPU as a minimum eigen solver
# See https://github.com/dwavesystems/dwave-qiskit-plugin
num_reads = 1000
cliqueSampler = DWaveCliqueSampler()
dwave_solver_clique = DWaveMinimumEigensolver(sampler = cliqueSampler, num_reads = num_reads)
optimizer = MinimumEigenOptimizer(dwave_solver_clique)


In [6]:
result = optimizer.solve(model) # Problem ID : f684f451-2f9d-4ebd-ad84-983ac989838b

In [7]:
print(result)

optimal function value: 1.0
optimal value: [2. 0. 0. 0. 1. 2.]
status: SUCCESS


In [8]:
result.min_eigen_solver_result.sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,chain_break_fraction,energy,num_occurrences
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.055556,-270254.5,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.055556,-269990.5,1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0.055556,-270253.5,4
3,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0.000000,-270569.5,2
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,1,0,0.055556,-270185.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,0,1,0,1,0,0,0,1,0,1,...,0,1,0,0,1,0,0,0.000000,-230758.5,1
718,1,1,0,0,1,0,0,1,0,1,...,0,0,1,0,1,0,0,0.000000,-230568.5,2
719,1,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0.055556,-249571.5,1
720,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0.111111,-266862.5,1


# Solve with Leaps Hybrid Solver

In [9]:
dwave_solver_hybrid = DWaveMinimumEigensolver(sampler =  LeapHybridSampler())
optimizer = MinimumEigenOptimizer(dwave_solver_hybrid)

In [10]:
result = optimizer.solve(model) # Problem ID : cef4cc6d-8686-4491-8eb7-3fdbba3b7ec2

In [11]:
print(result)

optimal function value: 1.0
optimal value: [2. 0. 0. 0. 1. 2.]
status: SUCCESS


In [12]:
result.min_eigen_solver_result.sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,energy,num_occurrences
0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,-270569.5,1
